### Importing Necessary Libraries

In [1]:
import sys
import argparse
from tern_rpe.tools.utils.getter import *
from tern_rpe.datasets import *
from tern_rpe.modules.metrics import *
from tern_rpe.datasets import *
from tern_rpe.modules.models import *
from tern_rpe.modules.losses import *
from tern_rpe.tools.trainer import *
from tern_rpe.datasets.augmentations import *
from tern_rpe.tools.loggers import *
from tern_rpe.tools.configs import *

import os
import cv2
import math
import json
from tqdm import tqdm
from datetime import datetime

import torch
import torch.nn as nn
import torch.utils.data as data
from torch.utils.data import DataLoader
import torchvision.models as models
from torch.optim import SGD, AdamW
from torch.optim.lr_scheduler import StepLR, CosineAnnealingLR, LambdaLR, ReduceLROnPlateau, OneCycleLR, CosineAnnealingWarmRestarts

from tern_rpe.tools.utils import download_pretrained_weights
from tern_rpe.tools.utils.cuda import NativeScaler, get_devices_info

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from tern_rpe.datasets.augmentations.transforms import MEAN, STD, get_resize_augmentation

from tern_rpe.tools.utils.random_seed import seed_everything

## Train

In [2]:
CACHE_DIR='./.cache'

def get_instance(config, **kwargs):
    # Inherited from https://github.com/vltanh/pytorch-template
    assert 'name' in config
    config.setdefault('args', {})
    if config['args'] is None:
        config['args'] = {}
    return globals()[config['name']](**config['args'], **kwargs)

def get_lr_policy(opt_config):
    optimizer_params = {}
    lr = opt_config['lr'] if 'lr' in opt_config.keys() else None
    if opt_config["name"] == 'sgd':
        optimizer = SGD
        optimizer_params = {
            'lr': lr, 
            'weight_decay': opt_config['weight_decay'],
            'momentum': opt_config['momentum'],
            'nesterov': True}
    elif opt_config["name"] == 'adam':
        optimizer = AdamW
        optimizer_params = {
            'lr': lr, 
            'eps': 1e-9,
            'weight_decay': opt_config['weight_decay'],
            'betas': (opt_config['momentum'], 0.98)}
    return optimizer, optimizer_params

def get_lr_scheduler(optimizer, lr_config, **kwargs):

    scheduler_name = lr_config["name"]
    step_per_epoch = False

    if scheduler_name == '1cycle-yolo':
        def one_cycle(y1=0.0, y2=1.0, steps=100):
            # lambda function for sinusoidal ramp from y1 to y2
            return lambda x: ((1 - math.cos(x * math.pi / steps)) / 2) * (y2 - y1) + y1

        lf = one_cycle(1, 0.2, kwargs['num_epochs'])  # cosine 1->hyp['lrf']
        scheduler = LambdaLR(optimizer, lr_lambda=lf)
        step_per_epoch = True
        
    elif scheduler_name == '1cycle':
        scheduler = OneCycleLR(
            optimizer,
            max_lr=0.001,
            epochs=kwargs['num_epochs'],
            steps_per_epoch=int(len(kwargs["trainset"]) / kwargs["batch_size"]),
            pct_start=0.1,
            anneal_strategy='cos', 
            final_div_factor=10**5)
        step_per_epoch = False
        

    elif scheduler_name == 'plateau':
        scheduler = ReduceLROnPlateau(
            optimizer,
            mode='min',
            factor=0.5,
            patience=1,
            verbose=False, 
            threshold=0.0001,
            threshold_mode='abs',
            cooldown=0, 
            min_lr=1e-8,
            eps=1e-08
        )
        step_per_epoch = True

    elif scheduler_name == 'cosine':
        scheduler = CosineAnnealingWarmRestarts(
            optimizer,
            T_0=kwargs['num_epochs'],
            T_mult=1,
            eta_min=0.0001,
            last_epoch=-1,
            verbose=False
        )
        step_per_epoch = False

    elif scheduler_name == 'cosine2':
        scheduler = CosineWithRestarts(
            optimizer, 
            T_max=kwargs['train_len'])
        step_per_epoch = False
        
    return scheduler, step_per_epoch

import torch
import numpy as np
# code from AllenNLP

class CosineWithRestarts(torch.optim.lr_scheduler._LRScheduler):
    """
    Cosine annealing with restarts.
    Parameters
    ----------
    optimizer : torch.optim.Optimizer
    T_max : int
        The maximum number of iterations within the first cycle.
    eta_min : float, optional (default: 0)
        The minimum learning rate.
    last_epoch : int, optional (default: -1)
        The index of the last epoch.
    """

    def __init__(self,
                 optimizer: torch.optim.Optimizer,
                 T_max: int,
                 eta_min: float = 0.,
                 last_epoch: int = -1,
                 factor: float = 1.) -> None:
        # pylint: disable=invalid-name
        self.T_max = T_max
        self.eta_min = eta_min
        self.factor = factor
        self._last_restart: int = 0
        self._cycle_counter: int = 0
        self._cycle_factor: float = 1.
        self._updated_cycle_len: int = T_max
        self._initialized: bool = False
        super(CosineWithRestarts, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        """Get updated learning rate."""
        # HACK: We need to check if this is the first time get_lr() was called, since
        # we want to start with step = 0, but _LRScheduler calls get_lr with
        # last_epoch + 1 when initialized.
        if not self._initialized:
            self._initialized = True
            return self.base_lrs

        step = self.last_epoch + 1
        self._cycle_counter = step - self._last_restart

        lrs = [
            (
                self.eta_min + ((lr - self.eta_min) / 2) *
                (
                    np.cos(
                        np.pi *
                        ((self._cycle_counter) % self._updated_cycle_len) /
                        self._updated_cycle_len
                    ) + 1
                )
            ) for lr in self.base_lrs
        ]

        if self._cycle_counter % self._updated_cycle_len == 0:
            # Adjust the cycle length.
            self._cycle_factor *= self.factor
            self._cycle_counter = 0
            self._updated_cycle_len = int(self._cycle_factor * self.T_max)
            self._last_restart = step

        return lrs

In [3]:
parser = argparse.ArgumentParser('Training Object Detection')
parser.add_argument('--print_per_iter', type=int, default=300, help='Number of iteration to print')
parser.add_argument('--val_interval', type=int, default=2, help='Number of epoches between valing phases')
parser.add_argument('--save_interval', type=int, default=1000, help='Number of steps between saving')
parser.add_argument('--resume', type=str, default=None,
                    help='whether to load weights from a checkpoint, set None to initialize')
parser.add_argument('--saved_path', type=str, default='./weights')
parser.add_argument('--top_k', type=int, default=10, help='top k validation')
parser.add_argument('--no_visualization', action='store_false', help='whether to visualize box to ./sample when validating (for debug), default=on')

torch.backends.cudnn.benchmark = True
torch.backends.cudnn.fastest = True
seed_everything()

def train(args, config):
    # os.environ['CUDA_VISIBLE_DEVICES'] = config.globals['gpu_devices']
    # num_gpus = len(config.globals['gpu_devices'].split(','))

    # device = torch.device("cuda" if torch.cuda.is_available() else 'cpu') ---> UNCOMMENT IF RUNNING ON CPU/GPU
    # devices_info = get_devices_info(config.globals['gpu_devices'])
    
    device = torch.device("mps:0") # For Metal M1 GPU
    
    trainloader = get_instance(config.trainloader, device=device)
    valloader = get_instance(config.valloader) 

    trainset = trainloader.dataset
    valset = valloader.dataset

    net = get_instance(config.model, device=device)

    optimizer, optimizer_params = get_lr_policy(config.lr_policy)

    criterion = get_loss_fn(config.loss)

    valset1 = get_instance(config.valset1)
    valset2 = get_instance(config.valset2)

    metric = RetrievalScore(
            valset1, valset2, 
            max_distance = 1.3,
            top_k=args.top_k,
            metric_names=["R@1", "R@5", "R@10"],
            dimension=config.model['args']['d_embed'],
            save_results=True)

    model = Retriever(
            model = net,
            metrics = metric,
            criterion=criterion,
            scaler=NativeScaler(),
            optimizer= optimizer,
            optim_params = optimizer_params,     
            device = device)

    if args.resume is not None:                
        load_checkpoint(model, args.resume)
        start_epoch, start_iter, best_value = get_epoch_iters(args.resume)
    else:
        print('Not resume. Load pretrained weights...')
        start_epoch, start_iter, best_value = 0, 0, 0.0
        
    scheduler, step_per_epoch = get_lr_scheduler(
        model.optimizer, train_len=len(trainloader),
        lr_config=config.lr_scheduler,
        num_epochs=config.globals['num_epochs'])

    if args.resume is not None:                 
        old_log = find_old_log(args.resume)
    else:
        old_log = None

    args.saved_path = os.path.join(
        args.saved_path, 
        datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))

    trainer = Trainer(config,
                     model,
                     trainloader, 
                     valloader,
                     checkpoint = Checkpoint(save_per_iter=args.save_interval, path = args.saved_path),
                     best_value=best_value,
                     logger = Logger(log_dir=args.saved_path, resume=old_log),
                     scheduler = scheduler,
                     evaluate_per_epoch = args.val_interval,
                     visualize_when_val = args.no_visualization,
                     step_per_epoch = step_per_epoch)
    print()
    print("##########   DATASET INFO   ##########")
    print("Trainset: ")
    print(trainset)
    print("Valset: ")
    print(valset)
    print()
    print(trainer)
    print()
    print(config)
    # print(f'Training with {num_gpus} gpu(s): ')
    # print(devices_info)
    print(f"Start training at [{start_epoch}|{start_iter}]")
    print(f"Current best R@10: {best_value}")

    trainer.fit(start_epoch = start_epoch, start_iter = start_iter, num_epochs=config.globals['num_epochs'], print_per_iter=args.print_per_iter)

In [4]:
sys.argv = ['']

In [5]:
args = parser.parse_args()
config = Config("./tern_rpe/tools/configs/yaml/tern.yaml")
train(args, config)

loading annotations into memory...
Done (t=0.08s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Not resume. Load pretrained weights...

##########   DATASET INFO   ##########
Trainset: 
Number of images: 29000
Number of texts: 145000

Valset: 
Number of images: 1014
Number of texts: 5070


##########   MODEL INFO   ##########
Model name: TERN
Number of trainable parameters:  26,404,096
Training iterations per epoch: 454
Validating iterations per epoch: 16

##########   CONFIGURATION INFO   ##########
trainloader:
  name:    NumpyFeatureLoader
  args:
    root_dir:      /Users/rishideychowdhury/Desktop/Joint-Embedding/Data/flickr30k_images/flickr30k_images/flickr30k_images
    ann_path:      /Users/rishideychowdhury/Desktop/Joint-Embedding/Data/flickr30k_

/Users/rishideychowdhury/Desktop/Joint-Embedding/env/lib/python3.8/site-packages/torch/cuda/amp/grad_scaler.py:118: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
/Users/rishideychowdhury/Desktop/Joint-Embedding/env/lib/python3.8/site-packages/pytorch_metric_learning/utils/loss_and_miner_utils.py:50: UserWarning: The operator 'aten::nonzero' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1665731529781/work/aten/src/ATen/mps/MPSFallback.mm:11.)
  a1_idx, p_idx = torch.where(matches)


[0|100] [300|45854] || T: 0.88436 || Time:    77.0980s
[1|100] [600|45854] || T: 0.41499 || Time:    34.8402s
[1|100] [900|45854] || T: 0.86021 || Time:    71.7410s
Save model at [1000|45854] to last.pth
[2|100] [1200|45854] || T: 0.83759 || Time:    69.0729s
=============================EVALUATION===================================


100%|██████████| 16/16 [00:21<00:00,  1.37s/it]


Extracting features...


100%|██████████| 317/317 [00:20<00:00, 15.73it/s] 


Calculating distance matrice...
Saving retrieval results...
Calculating distance matrice...


TypeError: object of type 'numpy.int64' has no len()

In [7]:
data = np.load('./results/i2t_results.npy', allow_pickle=True)

In [22]:
data[()]

{67: {'pred_ids': [10030,
   141432,
   84585,
   141433,
   141431,
   112279,
   48736,
   124525,
   131580,
   136972],
  'target_ids': array([335, 336, 337, 338, 339]),
  'scores': array([0.6468612 , 0.6535726 , 0.6599468 , 0.67073226, 0.67989707,
         0.67993665, 0.6810855 , 0.6827724 , 0.6930741 , 0.7012409 ],
        dtype=float32)},
 99: {'pred_ids': [51606,
   23205,
   66662,
   51106,
   100731,
   54913,
   95353,
   49291,
   2099,
   51608],
  'target_ids': array([495, 496, 497, 498, 499]),
  'scores': array([0.6181389 , 0.62308073, 0.6364105 , 0.64762425, 0.6480419 ,
         0.65864426, 0.6594637 , 0.6614181 , 0.6727432 , 0.6756002 ],
        dtype=float32)},
 100: {'pred_ids': [95352,
   129011,
   74491,
   74121,
   36910,
   74180,
   151550,
   119766,
   80008,
   64518],
  'target_ids': array([500, 501, 502, 503, 504]),
  'scores': array([0.6343535 , 0.6419978 , 0.64342546, 0.6443602 , 0.6472012 ,
         0.65119004, 0.6558116 , 0.65714693, 0.66084343, 0.66

In [23]:
len(data[()])

1014

In [26]:
len(data[()][67]['pred_ids'])

10

In [27]:
a = np.array([50])

In [34]:
len([a[0]])

1

In [35]:
type(np.int64(5) == np.int64

SyntaxError: unexpected EOF while parsing (3510347988.py, line 1)

In [37]:
isinstance([a[0]], np.int64)

False

[[50]]

In [42]:
[[a[0]]]

[[50]]

In [44]:
a.tolist()

[50]